<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Guided Problem Set 3: Fourier Analysis</h1>


<a name='section_3_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P3.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_3_1">P3.1 Frequency Analysis</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_3_1">P3.1 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_3_2">P3.2 The Discrete Fourier Transform</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_3_2">P3.2 Problems</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_3_3">P3.3 Spectrogram and Q-Transform</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#problems_3_3">P3.3 Problems</a></td>
    </tr>
</table>



<h3>Learning Objectives</h3>

The goal of this exercise is to prepare you for analyzing data in fourier space. These exercises will show you how to go from a simple set of frequencies to a full data analysis. As an example, we will use music, but this can easily be applied to any setup in frequency space.

In this problem set we will explore the following objectives
- Understand the frequency analysis of continuous and discrete time signals
- Visualize the connection between time and frequency space
- Characterize the energy/power carried by different frequencies
- Filter and denoise a signal by visualizing its spectral content


<h3>Importing Libraries</h3>

Before beginning, run the cell below to import the relevant libraries for this notebook. 
Optionally, set the plot resolution and default figure size.


In [ ]:
#>>>RUN
!pip install playsound
!pip install soundfile

In [ ]:
#>>>RUN

import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.io.wavfile import write

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure size
plt.rcParams['figure.figsize'] = (9,6)

<a name='section_3_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P3.1 Frequency Analysis</h2>    

| [Top](#section_3_0) | [Previous Section](#section_3_0) | [Problems](#problems_3_1) | [Next Section](#section_3_2) |


<h3>Motivation: Gravitational Waves/LIGO</h3>

Every massive object that accelerates produces gravitational waves. These gravitational waves can be intuitively understood as "ripples" in space-time that travel at the speed of light. One of the largest sources of continuous gravitational waves are produced by spinning massive objects such as neutron stars and black holes. Much can be learned about these objects by studying their gravitational wave signals so they are often studied at the gravitational wave detectors in the US (LIGO). 

Compact Binary Inspiral Gravitational Waves are of particular interest and show up as significant detections in LIGO. These waves are created by one of three sources: 

- Binary Neutron Stars (BNS)
- Binary Black Hole (BBH)
- Neutron Star-Black Hole Binary (NSBH)


<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/ligo.jpg" width="600"/>
</p>

When these events occur, they are measured on Earth at one of the LIGO detectors. When the gravity waves of a cataclysmic merger pass through the earth, they morph space-time changing the length of a 4km LIGO arm by a thousandth of the width of a proton. The LIGO detector is able to be sensitive to these very small changes and thus can generate strain vs. time data for the gravitation wave.

Strain is the instrument's detected space change within an arm in comparison to the total space (length) of the arm. In the event of a detectable merger this strain data will form a "gravitational wave signal".

The goal, then, of the LIGO groups is to filter out the noise of the signal and extract parameters from the signal itself using a template fit. These parameters often lead insight to the specific event or even the universe itself.


<h3>Frequency Analysis</h3>

<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/strain.png" width="800"/>
</p>

Often, the time-series data is too noisy/messy for easy analysis, so instead we can consider the underlying frequency representation of the same information. The allows us to concentrate on the frequencies for which we expect the black hole merger to exist and easily filter out frequencies that come from power lines, earthquake disturbances, or other noise sources.

If we are able to convert the time-series data to frequency based data, filter out the noise frequencies, and then convert the remaining signal to time-series data, we can get a 'cleaner' time-series version of the data. This cleaned version is then much easier to analyze in further research. 

This translation of time-series data to frequencies and vice versa are called fourier transforms and inverse fourier transforms respectively, and is the main focus of the problem set. 

**Fourier Transform**<br />
*Space or Time Functions* &rarr; *Spatial or Temporal Frequencies* <br />

**Inverse Fourier Transform**<br />
*Spatial or Temporal Frequencies* &rarr; *Space or Time Functions*

<h3>What do I mean by "Frequency Space"</h3>

In 1807 Joseph Fourier posited that any periodic signal could be represented by a sum of a particular set of harmonic sinusoids. 

In other words, all signals can be decomposed into elemental sine and cosine components, as follows:

$$f(t) = c_0 + \sum\limits_{k=1}^\infty (c_k\cos(k\omega_o t) + d_k\sin(k\omega_o t))$$

<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/decompose.gif" width="800"/>
</p>

<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/recompose.gif" width="800"/>
</p>

Convince yourself that the follwing sinusoidal decomposition is indeed the case.

$$e^{ix} = \cos(x) + i\sin(x)$$

<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/plane.png" width="400"/>
</p>

<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/sawtooth.gif" width="800"/>
</p>

<a name='problems_3_1'></a>     

| [Top](#section_3_0) | [Restart Section](#section_3_1) | [Next Section](#section_3_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem-3.1.1</span>

The code below plots a function in the space or time domain. What will it's frequency domain look like? Specifically, how many peaks will it have and will they have the same height?

Answer in the form of a list with the first element indicating how many peaks and the second element being 1 if all the peaks are the same height or 0 if not (e.g. `[2,0]` for two peaks with different heights).

*Hint: look at the function written in the code below. Try plotting this yourself! We will learn how to plot functions in the frequency domain in the next section.*


In [ ]:
#>>>PROBLEM
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

import numpy as np
import matplotlib.pyplot as plt

x = np.arange(0,4*np.pi,0.01)   # start,stop,step
y = np.sin(x) + np.sin(3*x) + np.sin(4*x) + np.sin(15*x) + np.sin(7*x) + .3*np.sin(20*x)

plt.plot(x, y)

#plot labels and style
plt.xlabel('x', fontsize=15) #Label x
plt.ylabel('y', fontsize=15)#Label y

# changing the fontsize of ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem-3.1.2</span>

Given the formula below, what is the maximum and minimum value of the signal amplitude? Report your result as a list of numbers `[max,min]` with precision 1e-1.

*Hint: look at the function written in the code below. Try plotting this yourself!*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.arange(0,4*np.pi,0.01)   # start,stop,step
y = np.sin(x) + np.sin(4*x) + np.sin(20*x)

plt.plot(x, y)

#plot labels and style
plt.xlabel('x', fontsize=15) #Label x
plt.ylabel('y', fontsize=15)#Label y

# changing the fontsize of ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem-3.1.3</span>

In the code above, if the sin(4x) became sin(3x), would the maximum be larger or smaller?

*Hint: Try plotting this yourself!*

<a name='section_3_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P3.2 The Discrete Fourier Transform</h2>    

| [Top](#section_3_0) | [Previous Section](#section_3_1) | [Problems](#problems_3_2) | [Next Section](#section_3_3) |


<h3>Overview</h3>

- These decomposed sinusoids are characterized by their frequencies.
- Want a mathematical machine that treats signals with a given frequency different than other frequencies. 


<h3>Analysis (Discrete Fourier Transform)</h3>

- We can inspect a signal and ask how much of one frequency is present.
- Multiply our signal by a sinusoid of a particular frequency to ask how "similar" the two are. 
- Call these the *Fourier frequency coefficients.*
$$ X[k] = \frac{1}{N}\sum\limits_{n=1}^{N-1} x[n]e^{-i\frac{2\pi k}{N} n} $$ 
<br>


<table style="width:50%">
    <colgroup>
       <col span="1" style="width: 50%;">
       <col span="1" style="width: 50%;">
    </colgroup>
    <tr>
        <th style="text-align: left; font-size: 13pt;">Syntax</th>
        <th style="text-align: left; font-size: 13pt;">Description</th>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">$ x[n]$</td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">Signal for analysis</td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">$ e^{-i\frac{2\pi k}{N} n}$</td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">Frequency component k we compare to</td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">$ X[k] $</td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;">How similar to the sinusoid of component k</td>
    </tr>
</table>


$$X[k] = \frac{1}{N}\sum\limits_{n=1}^{N-1} (e^{-i\frac{2\pi j}{N} n})(e^{i\frac{2\pi k}{N} n}) = \frac{N}{N}\delta_{jk} $$ 
<br>

***The "DC" ($X[0]$) Term***
- Is the 0 frequency component of a signal.
- Represents the constant offset of a signal (if any). 
- Represents the *average value* of a signal over one period. 


***Synthesis (Inverse Discrete Fourier Transform)***
- Can recompose signal with these coefficients.
$$x[n] = \sum\limits_{k=0}^{N-1}X[k]e^{i\frac{2\pi k}{N} n}$$


<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/box_transform.PNG" width="700"/>
</p>
<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/box_transform_2.PNG" width="700"/>
</p>

Notice that when we tightened our signal in time, we made it wider in frequency. This is a visual representation of the famous _uncertainity principle_! It arises in quantum mechanics because position and momentum obey the same Fourier relationship that time and frequency do above.

Take a look at the code below which uses numpy's built in `fft` method to take a discrete fourier tranform. Play around with the signal width and verify the uncertainty principle.

In [ ]:
#>>>RUN

import numpy as np
import matplotlib.pyplot as plt

samplingFrequency   = 100
signal_width = 2

#Plotting amplitude vs. time
x = np.arange(-5, 5, .01)
amplitude = np.piecewise(x, [x < 0-signal_width/2, ((x >= 0-signal_width/2) & (x < 0+signal_width/2)), x >= 0+signal_width/2], [0, 1, 0])
plt.title('Signal across time',fontsize=15)
plt.plot(x, amplitude)
plt.xlabel('Time',fontsize=15)
plt.ylabel('Amplitude',fontsize=15)
# changing the fontsize of ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

#Using np.fft.fft to take the fourier transform
fourierTransform = np.fft.fft(amplitude)/len(amplitude)    #Devide by length of the amplitude to normalize the transform
fourierTransform = fourierTransform[range(-int(len(amplitude)/2),int(len(amplitude)/2))] #Only look in a select range
    
#parametrize the frequency space
tpCount     = len(amplitude)
values      = np.arange(-int(tpCount/2), int(tpCount/2))
timePeriod  = tpCount/samplingFrequency
frequencies = values/timePeriod

#Plot the frequency space
plt.title('Fourier transform',fontsize=15)
plt.plot(frequencies, abs(fourierTransform))
plt.xlabel('Frequency',fontsize=15)
plt.ylabel('Amplitude',fontsize=15)
# changing the fontsize of ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()


<h3>Spectral Density/Power Spectrums</h3>


<h4>First the Energy Spectral Density (ESD)</h4>

- Describes how the energy of a signal is distributed with frequency. 
- The squared magnitude of each frequency 
- Good for localised transients (pulse-like signals) whose energy is concentrated around one time window. 

$$\overline{S}_{xx}(f) = |{\hat{x}(f)}|^2$$

<h4>Power Spectral Density (PSD)</h4>

- For continuous signals over all time, the power spectal density is more apt. 
- The power spectral density of a time series is the measure of the signal's power content in the frequency components that compose that signal.

$$S_{xx}(f) = \lim_{T \to \infty} \frac{1}{T}|\hat{x}_T(f)|^2 \ \textrm{where} \ x_T \ \textrm{is the signal, windowed}$$

$$S_{xx}(f) = \int_{-\infty}^{\infty} R_{xx}(\tau)e^{-i2\pi f\tau}d\tau = \hat{R}_{xx}(f) $$

- Defined as the normalized limit of the ESD for the windowed signal.
- Also represented as the Fourier transform of the autocorrelation function $R_{xx}(\tau)$ (informally, the Fourier transform of how similar the signal is to itself). 

<h4>Amplitude Spectral Density (ASD)</h4>

- Amplitude Spectral Density (ASD) is just the square-root of the PSD and is useful when the shape of the spectrum is rather constant, since variations in the ASD will then be proportional to variations in the signal's voltage itself.

<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/spectrum.png" width="500"/>
</p>


- Dominated by low frequencies and sharp spectral lines throughout. 
- From instrumental artifacts
    - Seismic noise
    - "Violin Modes" from the suspension fibers of the LIGO mirrors. 


<a name='problems_3_2'></a>     

| [Top](#section_3_0) | [Restart Section](#section_3_2) | [Next Section](#section_3_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem-3.2.1</span>

Consider the below `amplitude` function, which is simply a mixing of 3 different sinusoidal functions. Using `np.fft.fft`, translate the function from its time domain to its frequency domain. Make sure to normalize the fourier transform (as done in the previouis code segment).

What is the height of the tallest peak in the frequency space? Enter your answer as number with precision 1e-1.

In [ ]:
#>>>PROBLEM
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

samplingFrequency   = 100
samplingInterval = 1 / samplingFrequency

beginTime = 0
endTime = 10

time = np.arange(beginTime, endTime, samplingInterval);

amplitude1 = np.sin(2*np.pi*23*time)
amplitude2 = 3*np.sin(2*np.pi*17*time)
amplitude3 = -1.5*np.sin(2*np.pi*13*time)

amplitude = amplitude1 + amplitude2 + amplitude3
plt.title('Sine wave with multiple frequencies',fontsize=15)
plt.plot(time, amplitude)
plt.xlabel('Time',fontsize=15)
plt.ylabel('Amplitude',fontsize=15)
# changing the fontsize of ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

##############################################
############## INSERT CODE HERE ##############
##############################################


##############################################
##############################################

plt.title('Fourier transform depicting the frequency components',fontsize=15)
plt.grid(color='grey', linestyle='-', linewidth=1)
plt.plot(frequencies, abs(fourierTransform))
plt.xlabel('Frequency',fontsize=15)
plt.ylabel('Amplitude',fontsize=15)
# changing the fontsize of ticks
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem-3.2.2</span>

Compute the ESD and approximate the PSD and ASD of the above signal. What are the values of the ESD, PSD, and ASD at the frequncy $f_0=17$ Hz?

Hint: Consider the definitions previously given.

<pre>
ESD = np.abs(fourierTransform)**2
PSD = ESD/endTime
ASD = np.sqrt(PSD)
</pre>

Enter your answer as a list of numbers `[ESD(f0), PSD(f0), ASD(f0)]` with precision 1e-2.


Would it be easier to do this manually or with a computer?

<a name='section_3_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">P3.3 Spectrogram and Q-Transform</h2>   

| [Top](#section_3_0) | [Previous Section](#section_3_2) | [Problems](#problems_3_3) |


<h3>Overview</h3>

- To better understand a system, we can bridge the gap between the frequency and time domains with another transform.
- The Short-Time Fourier Transform takes the Fourier transform within shorter time segments. 
- The resulting visual representation is called a "spectrogram" which can be thought of as a series of Fourier transforms stacked on their side. 

<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/motif_sgram.png" width="400"/>
</p>

- The Q-Transform breaks these intervals up with logarithmic spacing for when the data are better represented as such (often times in audio.)

<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/stft_vs_q_gautham.png" width="400"/>
</p>
<!-- https://ccrma.stanford.edu/~gautham/Site/Multipitch.html -->

<h3>Whitening data</h3>

- From the ASD above, we can see that noise fluctuations are much larger at low and high frequencies and near spectral lines. 
- To better visualize deviations from the noise, it is useful to employ a technique called "whitening".
- Whitening takes the data and attempts to make the power spectral density flat (i.e. normalize the power at all frequencies) so that excess power at any frequency is more obvious. 

- Persisent signals like noise will have their power spread about the entire time window. 
- Localized signals will have all of their power in one region region (evident spikes)

- Achieved very informally by applying the inverse frequency response of the raw signal. 

Example: Trasmitted power in one of the interferoeter arms with two large glitches with a frequency around 5-50Hz

<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/whiteningexample_gwpy.png" width="800"/>
</p>


<h3>Filtering</h3>

- The amplification/attenuation of frequency components.
- Can be done in the time domian but best viewed from the frequency domain. 
    - **Multiplication in the frequency domain**
    - A set of scale factors H() which is refered to as the frequency response of the system.
    
<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/filters.png" width="800"/>
</p>


<h4>DFT Filtering</h4>

- We can filter using DFT coefficients by scaling them appropriately.

<h3>Example with Sound File</h3>

We are going to work through an example of filtering a sound file. First, we will import the relevant libraries and download a free sound file, below.

To read more about soundfile and sounddevice, see here:

- https://pysoundfile.readthedocs.io/en/latest/
- https://python-sounddevice.readthedocs.io/en/0.4.5/

In [ ]:
#>>>RUN
import time
import soundfile as sf
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import Audio, display

#first we load a sound from this link here https://freesound.org/s/647536/

data, samplerate = sf.read('data/P03/track.flac')

<h4>Amplitude and Frequency Manipulation</h4>

Working through the cells below, we will do the following:

- play the original file for 2 seconds
- play a note at 440 Hz
- Fourier transform the 440 Hz note and plot the frequency spectrum
- decrease the amplitude of the 440 Hz note in Fourier space, convert back to the time domain and play
- shift the frequency of the 440 Hz note in Fourier space (converting to 880 Hz), convert back to the time domain and play

In [ ]:
#>>>RUN

#play the original file
#the original file is 60s

def play(iArray,iFS):
    sf.write('data/P03/tmp.flac', iArray, iFS)
    display(Audio('data/P03/tmp.flac',autoplay=False))


play(data,samplerate)

In [ ]:
#>>>RUN

#play the original file for 2 seconds
#the original file is 60s

sec=2
min=0
max=int((sec/60.)*len(data))
play(data[min:max],samplerate)

In [ ]:
#>>>RUN

#play a note at 440 Hz
sec=2
timeseq = np.arange(0,sec,1./samplerate);

#lets do an A at 440Hz
a440 = np.sin(2*np.pi*440*timeseq)
play(a440,samplerate)

In [ ]:
#>>>RUN

#Fourier transform the 440 Hz note and plot the frequency spectrum
fta440 = np.fft.fft(a440)

values      = np.arange(int(len(a440)))
timePeriod  = len(a440)/samplerate
frequencies = values/timePeriod
plt.plot(frequencies[0:2000],np.abs(fta440[0:2000]))
plt.show()

In [ ]:
#>>>RUN

#decrease the amplitude of the 440 Hz note in Fourier space,
#convert back to the time domain and play
ifta440 = np.fft.ifft(fta440*0.1)

play(ifta440.real,samplerate)

In [ ]:
#>>>RUN

#shift the frequency of the 440 Hz note in Fourier space (converting to 880 Hz),
#convert back to the time domain and play
fta880  = fta440
fta880[880*2] = fta440[440*2]
fta880[440*2] = 0
ifta880 = np.fft.ifft(fta880)

play(ifta880.real,samplerate)

<h4>Bass Boosting</h4>

Now we will manipulate the data in Fourier space to increase the amplitude of the lower frequencies. We first plot the result of this manipulation in the frequency domain.

In [ ]:
#>>>RUN
length_sec=len(data)/samplerate
fc = 400 #cutoff frequency in Hz; boost all frequency content below fc
kc = int(fc*length_sec)

X = np.fft.fft(data[:,0])
Y = X.copy() #copy values from X
#boost the bass
Y[:kc]    = [i*10 for i in X[:kc]]

def plotFFT(X,Y,data,samplerate):
    values      = np.arange(int(len(data[:,0])))
    timePeriod  = len(data[:,0])/samplerate
    frequencies = values/timePeriod
    plt.plot(frequencies[0:int(length_sec*2000)],np.abs(X[0:int(length_sec*2000)]),alpha=0.5,label='before')
    plt.plot(frequencies[0:int(length_sec*2000)],np.abs(Y[0:int(length_sec*2000)]),alpha=0.5,label='after')
    plt.ylabel('amp')
    plt.xlabel('freq')
    plt.legend() 
    plt.yscale('log')
    plt.show()

plotFFT(X,Y,data,samplerate)

Now we take the inverse Fourier transform and play the audio to compare with the original.

In [ ]:
#>>>RUN

## Inverse fourier transform and recombine into a .wav file 
y = np.fft.ifft(Y)

#start the file at 30 sec
def replay(y,data,samplerate=samplerate,playtime=10):
    print("play 1st: altered waveform")
    play(y[30*samplerate:(30+playtime)*samplerate].real,samplerate)
    
    print("play 2nd: original")
    #and for comparison
    play(data[30*samplerate:(30+playtime)*samplerate].real,samplerate)


replay(y,data)

<h4>Bass Isolation (Low Pass)</h4>

Finally, we use a low pass filter to truncate the data in the frequency domain. Again, the plot below shows the waveform in Fourier space, and then the data are played through the system audio.

In [ ]:
#>>>RUN
fc = 1000 #cutoff frequency in Hz; boost all frequency content below fc
kc = int(fc*length_sec)

X = np.fft.fft(data[:,0])
Y = X.copy() #copy values from X
Y[:kc]    = [i*10 for i in X[:kc]]
Y[kc:]    = [i*0 for i in X[kc:]]
y = np.fft.ifft(Y)

plotFFT(X,Y,data,samplerate)
replay(y,data,samplerate)

<h4>Signal Detection and Matched Filtering</h4>

- If we have an unknown, noisy signal we can try to detect the presence of a known signal with matched filtering. 
- If we *know* or *guess* the signal we're looking for (called the *template*), we can use it as a filter for combing the data for the presence of that template.
- Matched filters work by maximizing the signal to noise ratio (SNR) when the matched filter detects the presence of the template signal in a noisy signal. 
- We will talk more about convolutions next week but we can informally think of a matched filter as:
- "Drag" or sweep your template across the signal and calculate some statistic.
- The optimal statistic suggests the presence of a signal. 

<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/template.png" width="800"/>
</p>
<p align="center">
<img src="https://raw.githubusercontent.com/mitx-8s50/images/main/P03/SNR.png" width="800"/>
</p>


In [ ]:
#>>>RUN

dft_cof = np.fft.fft(data[:,0])
values      = np.arange(int(len(data[:,0])))
timePeriod  = len(data[:,0])/samplerate
frequencies = values/timePeriod
minHz=900
maxHz=1000
plt.plot(frequencies[int(length_sec*minHz):int(length_sec*maxHz)],np.abs(X[int(length_sec*minHz):int(length_sec*maxHz)]),alpha=0.5)
plt.ylabel('amp')
plt.xlabel('freq')
plt.legend() 
plt.show()


In [ ]:
#now lets isolate a frequency
minHz=0
maxHz=int(len(dft_cof)/int(length_sec))
isofreq=750
freqrange=50
dft_cof = np.fft.fft(data[:,0])
norm=np.sum(dft_cof)
for tmpfreq in range(minHz*int(length_sec),maxHz*int(length_sec)):
    if (float(tmpfreq) < (isofreq-freqrange)*length_sec) or (float(tmpfreq) > (isofreq+freqrange)*length_sec):
        dft_cof[tmpfreq] *= 0
scale=np.sum(dft_cof)/norm
dft_cof*=scale
plotFFT(X,dft_cof,data,samplerate)
isofreqdata = np.fft.ifft(dft_cof)
x           = np.fft.ifft(X)

replay(isofreqdata,x)

<a name='problems_3_3'></a>   

| [Top](#section_3_0) | [Restart Section](#section_3_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem-3.3.1</span>

So in this case, we are going to go to our sound file and modify the song so that amplitude across each range is exactly the same. This a varaition of what we call whitening. Go ahead and set this up.

**PHIL DETAIL HERE:**

In [ ]:
#>>>PROBLEM
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.
length_sec=len(data)/samplerate
dft_cof = np.fft.fft(data[:,0])

plotFFT(X,dft_cof,data,samplerate)
isofreqdata = np.fft.ifft(dft_cof)
X           = np.fft.fft(data[:,0])
x           = np.fft.ifft(X)
replay(isofreqdata,x,playtime=16)


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Problem-3.3.2</span>

Now inject a sine wave of 440 Hz, by adding it to the signal. Using an amplitude of 1000, show how whitening (without the signal injected) makes our signal clear.

**PHIL DETAIL HERE:**

In [ ]:
length_sec=len(data)/samplerate
data_injected = data[:,0].copy()
timeseq = np.arange(0,length_sec,1./samplerate);
a440 = (np.sin(2*np.pi*440*timeseq + np.pi))*0.01
data_injected+= a440 
dft_cof     = np.fft.fft(data[:,0])
dft_cof_inj = np.fft.fft(data_injected)
dft_cof_inj_n = dft_cof_inj / np.abs(dft_cof)
#dft_cof_inj_n*=50
isofreqdata = np.fft.ifft(dft_cof_inj_n)
#Now we should scale up the volume so its loud enough
plotFFT(dft_cof_inj,dft_cof_inj_n,data,samplerate)
replay(data_injected,isofreqdata,playtime=16)
